In [1]:
import requests, pandas as pd, time
from newspaper import Article
from concurrent.futures import ThreadPoolExecutor, as_completed, ProcessPoolExecutor
from tqdm import tqdm

BASE_URL = "https://www.hankyung.com/article/"
HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                   "AppleWebKit/537.36 (KHTML, like Gecko) "
                   "Chrome/127.0.0.0 Safari/537.36"),
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.8",
    "Referer": "https://www.hankyung.com/",
}

def crawl_url(url):
    """단일 URL 크롤링: AMP 먼저 요청, 없으면 일반 페이지 시도"""
    try:
        # 1. AMP 페이지 먼저 시도
        amp = url.rstrip("/") + "/amp"
        r = requests.get(amp, headers=HEADERS, timeout=12)

        # 2. AMP 없으면 일반 페이지 시도
        if r.status_code == 404:
            r = requests.get(url, headers=HEADERS, timeout=12)

        # 3. 여전히 에러면 None
        if r.status_code in (403, 404):
            return None
        r.raise_for_status()

        # 4. newspaper3k 파싱
        art = Article(url, language='ko')
        art.set_html(r.text)
        art.parse()
        if not art.title or not art.text:
            return None

        return {
            "url": url,
            "title": art.title,
            "text": art.text,
            "publish_date": art.publish_date
        }
    except Exception:
        return None

def crawl_gen_articles_range(date_str, start_id, end_id, *,  # end_id는 '미포함' (파이썬 range 규칙)
                             max_workers=36, batch_size=1000, sleep_between=0.05,
                             save_path=None):
    """
    일반 기사 크롤링: [start_id, end_id) 구간만
      - AMP만 요청
      - 배치 제출
    """
    results, collected = [], 0
    total = end_id - start_id

    with ThreadPoolExecutor(max_workers=max_workers) as ex:
        with tqdm(total=total, desc=f"{date_str} [{start_id:05}-{end_id-1:05}]") as pbar:
            for start in range(start_id, end_id, batch_size):
                end = min(start + batch_size, end_id)
                urls = [f"{BASE_URL}{date_str}{num:05}" for num in range(start, end)]
                futures = [ex.submit(crawl_url, u) for u in urls]

                for fut in as_completed(futures):
                    res = fut.result()
                    if res:
                        results.append(res)
                        collected += 1
                    pbar.set_postfix_str(f"수집 {collected}")
                    pbar.update(1)
                    time.sleep(sleep_between)

    df = pd.DataFrame(results).drop_duplicates(subset=["url"])
    if save_path:
        df.to_csv(save_path, index=False)
    tqdm.write(f"[{start_id:05}-{end_id-1:05}] 최종 수집: {len(df)}")
    return df

In [2]:
day_str = "20250820"
start_id, end_id = 0, 20000   # 이 부분만 각 노트북에서 바꿔서 실행
out_path = f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{day_str}_{start_id:05}-{end_id-1:05}.csv"

df_part = crawl_gen_articles_range(
    day_str, start_id, end_id,
    max_workers=36, batch_size=1000, sleep_between=0.05,
    save_path=out_path
)

df_part

20250820 [00000-19999]: 100%|██████████| 20000/20000 [19:04<00:00, 17.47it/s, 수집 128]

[00000-19999] 최종 수집: 128


,url,title,text,publish_date
0,https://www.hankyung.com/article/2025082000057,"67세 마돈나, '38살 연하' 연인과 이탈리아서 화려한 생일파티","마돈나, 자메이카 출신 29세 연인 모리스와 생일 보내\n\n출처=마돈나 인스타그램...",2025-08-20 09:14:39+09:00
1,https://www.hankyung.com/article/2025082000044,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
2,https://www.hankyung.com/article/2025082000107,"안철수, 복당 1순위 '이준석' 선택 유일…李 ""옳고 강한 분""",'복당 1순위 홍준표? 이준석?' 질문에\n\n당권 주자 중 혼자 '이준석' 꼽은 ...,2025-08-20 09:39:32+09:00
3,https://www.hankyung.com/article/2025082000156,"코스피, 장 초반 2%대로 낙폭 확대…시총 상위주 일제히 '파란불'",20일 서울 중구 하나은행 본점 딜링룸 스크린에 지수가 표시돼 있다. /사진=연합뉴...,2025-08-20 09:24:05+09:00
4,https://www.hankyung.com/article/2025082000167,"[속보] 특검, 김건희 구속기간 이달 31일까지로 연장","[속보] 김여정 ""리재명, 역사의 흐름 바꿀 위인 아냐""\n\n김여정 북한 노동당 ...",2025-08-20 08:59:57+09:00
...,...,...,...,...
123,https://www.hankyung.com/article/2025082008617,"'석유화학산업 재도약'…구윤철 부총리 ""사즉생 각오로 위기 극복해야""","""10개사 최대 370만t 감축""\n\n구윤철 경제부총리 겸 기획재정부 장관이 20...",2025-08-20 13:36:45+09:00
124,https://www.hankyung.com/article/2025082008574,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
125,https://www.hankyung.com/article/2025082008604,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
126,https://www.hankyung.com/article/2025082008894,한국경제,트럼프 대통령이 푸틴 러시아 대통령과 만났습니다. 회담은 뉴욕 증시가 마감할 무렵 ...,NaT
